In [325]:
import pandas as pd
from functools import reduce
from operator import concat
from pandas.io.json import json_normalize

In [258]:
arquivo=r'C:\Users\gpira\Downloads\bq-results-20240515-184938-1715799987947.json'

In [259]:
df=pd.read_json(arquivo,lines=True)
df

,cnpj,area,grauProcesso,comarca,julgamento,dataDecisao,dataEncerramento,uf,tribunal,ultimoEstado,orgaoJulgador,citacaoTipo,unidadeOrigem,juiz,assuntosCNJ,partes,valorCausa,valorPredicaoCondenacao
0,41909082000179,Civel,1,Cuiaba - MT,Improcedência,2018-08-14T17:04:00Z,2018-09-11T17:40:00Z,MT,TJMT,Arquivado,- Cuiaba - MT,Citação por correio,Cuiaba,Adauto dos Santos Reis,[],[{'nomeNormalizadoNeoway': 'CARLOS ALBERTO ARA...,7000.00,3458.392820
1,44060029000134,Civel,1,Cuiaba - MT,Homologação de Decisão de Juiz Leigo,2017-10-30T14:36:00Z,2018-06-12T13:42:00Z,MT,TJMT,Arquivado,- Cuiaba - MT,Citação por correio,Cuiaba,Maria Aparecida Ferreira Fago,[],[{'nomeNormalizadoNeoway': 'DANIELLY BENEDITA ...,18740.00,5173.107959
2,46778443000109,Civel,1,Cuiaba - MT,Não identificado,NaN,2019-08-21T06:47:00Z,MT,TJMT,Arquivado,- Cuiaba - MT,Citação por correio,Cuiaba,Luis Aparecido Bertolucci Junior,[],[{'nomeNormalizadoNeoway': 'DANIEL GONCALVES D...,11019.99,4349.583422
3,45615485000166,Civel,1,Cuiaba - MT,Acordo homologado,2017-12-19T10:14:00Z,2018-01-23T01:03:00Z,MT,TJMT,Arquivado,- Cuiaba - MT,Citação por correio,Cuiaba,Juiz Titular 2,[],[{'nomeNormalizadoNeoway': 'ROSALIA DE SOUZA S...,564.56,3268.815161
4,26389632000182,Civel,1,Cuiaba - MT,Procedência em Parte,2017-02-13T21:18:00Z,2017-03-21T17:43:00Z,MT,TJMT,Arquivado,- Cuiaba - MT,Citação por correio,Cuiaba,Goncalo Antunes de Barros Neto,[],[{'nomeNormalizadoNeoway': 'GLEICIELE FERREIRA...,595.90,3268.815161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,41287540000185,Civel,1,Aracaju - SE,Julgamento,2020-03-18T15:23:16Z,2021-01-13T15:43:14Z,SE,TJSE,Arquivado,- Aracaju - SE,Sem citação,Aracaju,NaN,"[{'codigoCNJ': '50064', 'codigoLocal': '50064'...",[{'nomeNormalizadoNeoway': 'JORGE LUIZ DE SANT...,10012.90,3061.103148
19996,34198053000180,Civel,1,Aracaju - SE,Julgamento,2017-11-09T19:03:11Z,2019-05-29T14:30:25Z,SE,TJSE,Arquivado,- Aracaju - SE,Sem citação,Aracaju,NaN,"[{'codigoCNJ': '50064', 'codigoLocal': '50064'...",[{'nomeNormalizadoNeoway': 'MARIA ESTHER COSTA...,10919.19,3714.331870
19997,45750863000114,Civel,1,Aracaju - SE,Julgamento,2017-06-13T10:00:47Z,2018-02-02T14:32:37Z,SE,TJSE,Sentença proferida,- Aracaju - SE,Citação eletrônica,Aracaju,NaN,"[{'codigoCNJ': '50064', 'codigoLocal': '50064'...",[{'nomeNormalizadoNeoway': 'SIDNEY LEONARDO DO...,20179.96,2987.705051
19998,19586360000145,Civel,1,Lagarto - SE,Julgamento,2016-10-11T20:18:38Z,2017-09-19T13:41:10Z,SE,TJSE,Arquivado,- Lagarto - SE,Sem citação,Lagarto,NaN,"[{'codigoCNJ': '50064', 'codigoLocal': '50064'...",[{'nomeNormalizadoNeoway': 'CAROLINE NASCIMENT...,5136.80,2158.813944


In [262]:
#Alguns campos estão aninhados
#precisamos ver quais as condições 

df.assuntosCNJ.isna().unique()
#não foi indicado como nulo, pois são listas vazias


array([False])

In [348]:
#criação de função para identifcar o tamanho da lista dentro do campo aninhado assuntosCNJ

def itens_lista(df,nome_coluna,nome_coluna_final):
    
    """ conta quantos itens tem na coluna aninhada, isso serve para entendermos quais campos temos dentro dos dicionarios
    Args:
    df: O DataFrame Pandas.
    column_name: O nome da coluna que contém as listas.

  Returns:
    Uma nova coluna no DataFrame com o nome "nome_coluna_final" contendo o número de itens em cada lista.
  """
    
    df[nome_coluna_final] = df[nome_coluna].apply(len)
    return df
    
    
    

In [265]:
df=itens_lista(df,'assuntosCNJ','item_count')
df

,cnpj,area,grauProcesso,comarca,julgamento,dataDecisao,dataEncerramento,uf,tribunal,ultimoEstado,orgaoJulgador,citacaoTipo,unidadeOrigem,juiz,assuntosCNJ,partes,valorCausa,valorPredicaoCondenacao,item_count
0,41909082000179,Civel,1,Cuiaba - MT,Improcedência,2018-08-14T17:04:00Z,2018-09-11T17:40:00Z,MT,TJMT,Arquivado,- Cuiaba - MT,Citação por correio,Cuiaba,Adauto dos Santos Reis,[],[{'nomeNormalizadoNeoway': 'CARLOS ALBERTO ARA...,7000.00,3458.392820,0
1,44060029000134,Civel,1,Cuiaba - MT,Homologação de Decisão de Juiz Leigo,2017-10-30T14:36:00Z,2018-06-12T13:42:00Z,MT,TJMT,Arquivado,- Cuiaba - MT,Citação por correio,Cuiaba,Maria Aparecida Ferreira Fago,[],[{'nomeNormalizadoNeoway': 'DANIELLY BENEDITA ...,18740.00,5173.107959,0
2,46778443000109,Civel,1,Cuiaba - MT,Não identificado,NaN,2019-08-21T06:47:00Z,MT,TJMT,Arquivado,- Cuiaba - MT,Citação por correio,Cuiaba,Luis Aparecido Bertolucci Junior,[],[{'nomeNormalizadoNeoway': 'DANIEL GONCALVES D...,11019.99,4349.583422,0
3,45615485000166,Civel,1,Cuiaba - MT,Acordo homologado,2017-12-19T10:14:00Z,2018-01-23T01:03:00Z,MT,TJMT,Arquivado,- Cuiaba - MT,Citação por correio,Cuiaba,Juiz Titular 2,[],[{'nomeNormalizadoNeoway': 'ROSALIA DE SOUZA S...,564.56,3268.815161,0
4,26389632000182,Civel,1,Cuiaba - MT,Procedência em Parte,2017-02-13T21:18:00Z,2017-03-21T17:43:00Z,MT,TJMT,Arquivado,- Cuiaba - MT,Citação por correio,Cuiaba,Goncalo Antunes de Barros Neto,[],[{'nomeNormalizadoNeoway': 'GLEICIELE FERREIRA...,595.90,3268.815161,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,41287540000185,Civel,1,Aracaju - SE,Julgamento,2020-03-18T15:23:16Z,2021-01-13T15:43:14Z,SE,TJSE,Arquivado,- Aracaju - SE,Sem citação,Aracaju,NaN,"[{'codigoCNJ': '50064', 'codigoLocal': '50064'...",[{'nomeNormalizadoNeoway': 'JORGE LUIZ DE SANT...,10012.90,3061.103148,1
19996,34198053000180,Civel,1,Aracaju - SE,Julgamento,2017-11-09T19:03:11Z,2019-05-29T14:30:25Z,SE,TJSE,Arquivado,- Aracaju - SE,Sem citação,Aracaju,NaN,"[{'codigoCNJ': '50064', 'codigoLocal': '50064'...",[{'nomeNormalizadoNeoway': 'MARIA ESTHER COSTA...,10919.19,3714.331870,3
19997,45750863000114,Civel,1,Aracaju - SE,Julgamento,2017-06-13T10:00:47Z,2018-02-02T14:32:37Z,SE,TJSE,Sentença proferida,- Aracaju - SE,Citação eletrônica,Aracaju,NaN,"[{'codigoCNJ': '50064', 'codigoLocal': '50064'...",[{'nomeNormalizadoNeoway': 'SIDNEY LEONARDO DO...,20179.96,2987.705051,6
19998,19586360000145,Civel,1,Lagarto - SE,Julgamento,2016-10-11T20:18:38Z,2017-09-19T13:41:10Z,SE,TJSE,Arquivado,- Lagarto - SE,Sem citação,Lagarto,NaN,"[{'codigoCNJ': '50064', 'codigoLocal': '50064'...",[{'nomeNormalizadoNeoway': 'CAROLINE NASCIMENT...,5136.80,2158.813944,4


In [327]:
"""conseguimos identificar os nulos agora pela coluna item_count
    Vamos fazer um filtro
    e selecionar apenas os que contem 1 ou mais itens"""

df_filtrado=df.loc[df['item_count']>0]
df_filtrado['assuntosCNJ']


97       [{'titulo': 'Inclusao Indevida em Cadastro de ...
98       [{'titulo': 'Inclusao Indevida em Cadastro de ...
99       [{'titulo': 'Inclusao Indevida em Cadastro de ...
100          [{'titulo': 'Indenizacao por Dano Material'}]
101                         [{'titulo': 'Perdas e Danos'}]
                               ...                        
19995    [{'codigoCNJ': '50064', 'codigoLocal': '50064'...
19996    [{'codigoCNJ': '50064', 'codigoLocal': '50064'...
19997    [{'codigoCNJ': '50064', 'codigoLocal': '50064'...
19998    [{'codigoCNJ': '50064', 'codigoLocal': '50064'...
19999    [{'codigoCNJ': '50064', 'codigoLocal': '50064'...
Name: assuntosCNJ, Length: 19903, dtype: object

In [328]:
#pegamos os indices e utilizamos um for para passar linha a linha do novo df

indexes_df=df_filtrado.index

lista_parametros=[]
for i in range(len(indexes_df)):
    
    lista_parametros.append(list(df_filtrado['assuntosCNJ'][indexes_df[i]][0].keys()))

In [330]:
#identificamos quais são as chaves para extrairmos

lista_unica = reduce(concat, lista_parametros)

print(set(lista_unica)) # Saída: [1, 2, 3, 4, 5, 6, 7, 8, 9]

{'titulo', 'codigoLocal', 'codigoCNJ'}


In [350]:
#função de contagem de itens da lista para identificar quais casos vamos utilizar
df=itens_lista(df,'partes','cont_partes')

In [351]:
#filtro
df_filtrado2=df.loc[df['cont_partes']>0]
df_filtrado2['partes']

0        [{'nomeNormalizadoNeoway': 'CARLOS ALBERTO ARA...
1        [{'nomeNormalizadoNeoway': 'DANIELLY BENEDITA ...
2        [{'nomeNormalizadoNeoway': 'DANIEL GONCALVES D...
3        [{'nomeNormalizadoNeoway': 'ROSALIA DE SOUZA S...
4        [{'nomeNormalizadoNeoway': 'GLEICIELE FERREIRA...
                               ...                        
19995    [{'nomeNormalizadoNeoway': 'JORGE LUIZ DE SANT...
19996    [{'nomeNormalizadoNeoway': 'MARIA ESTHER COSTA...
19997    [{'nomeNormalizadoNeoway': 'SIDNEY LEONARDO DO...
19998    [{'nomeNormalizadoNeoway': 'CAROLINE NASCIMENT...
19999    [{'nomeNormalizadoNeoway': 'MARIA GENILDA DOS ...
Name: partes, Length: 19990, dtype: object

In [353]:
#pegamos os indices para passarmos linha a linha do df novo
indexes=df_filtrado2.index
#passamos linha a linha e salvamos tudo em uma lista para conferir posteriormente
lista_parametros_partes=[]
for i in range(len(df_filtrado2)):
    
    
    lista_parametros_partes.append(list(df_filtrado2['partes'][indexes[i]][1].keys()))

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            19990, 19991, 19992, 19993, 19994, 19995, 19996, 19997, 19998,
            19999],
           dtype='int64', length=19990)

In [361]:
#conferimos valores unicos

lista_unica = reduce(concat, lista_parametros_partes)

print(set(lista_unica)) # Saída: [1, 2, 3, 4, 5, 6, 7, 8, 9]

{'polo', 'nomeNormalizadoNeoway'}


In [362]:
#temos 2 dicionarios dentro da lista
#então para puxarmos os dados precisamos de duas funções 
df_filtrado2['partes'][19996]

[{'nomeNormalizadoNeoway': 'MARIA ESTHER COSTA CAMPOS DE SANTANA',
  'polo': 'ativo'},
 {'nomeNormalizadoNeoway': 'AZUL LINHAS AEREAS BRASILEIRAS',
  'polo': 'passivo'}]

In [366]:
#funções para extrair da coluna assuntoscnj

def extrair_titulo(row):
    try:
        return row.assuntosCNJ[0]['titulo']
    except (IndexError, KeyError):
        return ' '
def extrair_codigolocal(row):
    try:
        return row.assuntosCNJ[0]['codigoLocal']
    except (IndexError, KeyError):
        return ' '
def extrair_codigocnj(row):
    try:
        return row.assuntosCNJ[0]['codigoCNJ']
    except (IndexError, KeyError):
        return ' '

In [367]:
#funções para extrair da coluna partes
def extrai_partes_nome1(row):
    try:
        return row.partes[0]['nomeNormalizadoNeoway']
    except (IndexError, KeyError):
        return ' '
def extrai_partes_polo1(row):
    try:
        return row.partes[0]['polo']
    except (IndexError, KeyError):
        return ' '
def extrai_partes_nome2(row):
    try:
        return row.partes[1]['nomeNormalizadoNeoway']
    except (IndexError, KeyError):
        return ' '
def extrai_partes_polo2(row):
    try:
        return row.partes[1]['polo']
    except (IndexError, KeyError):
        return ' '

In [368]:
#adicionando os casos as colunas especificas

df.loc[:, 'tituloCNJ'] = df.apply(extrair_titulo, axis=1)
df.loc[:, 'codigolocalCNJ'] = df.apply(extrair_codigolocal, axis=1)
df.loc[:, 'codigoCNJ'] = df.apply(extrair_codigocnj, axis=1)
df.loc[:, 'nomeNormalizadoNeowayAtivo'] = df.apply(extrai_partes_nome1, axis=1)
df.loc[:, 'PoloAtivo'] = df.apply(extrai_partes_polo1, axis=1)
df.loc[:, 'nomeNormalizadoNeowayPassivo'] = df.apply(extrai_partes_nome2, axis=1)
df.loc[:, 'PoloPassivo'] = df.apply(extrai_partes_polo2, axis=1)

In [369]:
#removendo as colunas com dados aninhados
df=df.drop(columns=['partes','assuntosCNJ','item_count','cont_partes'])

In [370]:
df.to_csv(r'C:\Users\gpira\Downloads\Processos_judiciais_empresas.csv',header=True,index=False)